In [3]:
import tkinter as tk
from tkinter import filedialog
from collections import defaultdict
import numpy as np
from ultralytics import YOLO
from PIL import Image, ImageTk
import cv2

# 빨간색 범위 설정 (BGR 색 공간)
lower_red = np.array([0, 0, 100])
upper_red = np.array([60, 60, 255])
all_tracks = []
track_history = defaultdict(lambda: [])
frame_bounding_box_count = []
class VideoPlayerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("데모 시스템")
        self.root.geometry("1300x800")

        self.model = YOLO('C:/jupyter/best.pt')
        self.track_history = defaultdict(lambda: [])

        self.cap = None
        self.out = None
        self.canvas = None
        self.canvas_right = None  # 오른쪽 캔버스 추가
        self.frame_height = None
        self.right_video = None  # 오른쪽 캔버스에 표시되는 영상
        self.left_playing = False  # 왼쪽 영상 재생 상태 변수
        self.right_playing = False  # 오른쪽 영상 재생 상태 변수

        # 전체 프레임 생성
        self.frame = tk.Frame(self.root)
        self.frame.pack(expand=True, fill="both")

        # UI 요소 프레임 생성
        ui_frame = tk.Frame(self.frame)
        ui_frame.pack(side=tk.TOP, pady=20)

        title_label = tk.Label(ui_frame, text="데모 시스템", font=("Helvetica", 24))
        title_label.pack(pady=10)

        instruction_label = tk.Label(ui_frame, text="환자의 정자 영상을 업로드하세요")
        instruction_label.pack(pady=10)

        upload_button = tk.Button(ui_frame, text="영상 업로드", command=self.upload_video)
        upload_button.pack(pady=10)

        select_button = tk.Button(ui_frame, text="다른 영상 선택", command=self.select_video)
        select_button.pack(pady=10)

        # 왼쪽 캔버스를 생성
        self.canvas = tk.Canvas(self.frame, width=640, height=480)
        self.canvas.pack(side=tk.LEFT)

        # 오른쪽 캔버스를 생성
        self.canvas_right = tk.Canvas(self.frame, width=640, height=480)
        self.canvas_right.pack(side=tk.LEFT)

        # 왼쪽 영상 재생 상태를 제어할 버튼 추가
        play_button = tk.Button(ui_frame, text="왼쪽 영상 재생/일시정지", command=self.toggle_left_play)
        play_button.pack(pady=10)

        # 오른쪽 영상 재생 상태를 제어할 버튼 추가
        right_play_button = tk.Button(ui_frame, text="오른쪽 영상 재생/일시정지", command=self.toggle_right_play)
        right_play_button.pack(pady=10)

    def upload_video(self):
        file_path = filedialog.askopenfilename(filetypes=[("비디오 파일", "*.mp4")])
        if file_path:
            self.process_video(file_path)
            self.update_canvas()

    def select_video(self):
        file_path = filedialog.askopenfilename(filetypes=[("비디오 파일", "*.mp4")])
        if file_path:
            self.process_right_canvas(file_path)
            self.update_right_canvas()  # 오른쪽 캔버스 업데이트

    def process_video(self, video_path):
        self.cap = cv2.VideoCapture(video_path)
        frame_width = int(self.cap.get(3))
        self.frame_height = int(self.cap.get(4))
        self.out = cv2.VideoWriter("output_tracking12.avi", cv2.VideoWriter_fourcc(*'mp4v'), 17, (frame_width, self.frame_height))

    def process_right_canvas(self, video_path):
        self.right_video = cv2.VideoCapture(video_path)

    def update_canvas(self):
        if self.left_playing:
            ret, frame = self.cap.read()
            if not ret:
                self.cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                ret, frame = self.cap.read()  # 다시 처음으로 돌아감
                

            bounding_box_count = 0
            results = self.model.track(frame, persist=True, hide_labels=True)

            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = self.track_history[track_id]
                track.append((float(x), float(y)))
                if len(track) > 17:
                    track.pop(0)

                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(frame, [points], isClosed=False, color=(0, 0, 255), thickness=1)

                x1, y1, w, h = map(int, [x - w / 2, y - h / 2, w, h])
                cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 255, 0), 1)
                bounding_box_count += 1
                
            # 각 프레임의 바운딩 박스 수를 저장
            frame_bounding_box_count.append(bounding_box_count)
                
            red_mask = cv2.inRange(frame, lower_red, upper_red)
            red_only = cv2.bitwise_and(frame, frame, mask=red_mask)
            gray_image = cv2.cvtColor(red_only, cv2.COLOR_BGR2GRAY)
            _, binary_image = cv2.threshold(gray_image, 1, 255, cv2.THRESH_BINARY)
            contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            red_line_count = 0
            for contour in contours:
                contour_length = cv2.arcLength(contour, True)
                if contour_length >= 20:
                    red_line_count += 1
                    
            tracking_text = f"Tracking: {red_line_count}"
            cv2.putText(frame, tracking_text, (20, self.frame_height - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
        

            cv2.putText(frame, f"Detecting: {bounding_box_count}", (450, self.frame_height - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
             # 각 프레임의 바운딩 박스 수를 저장
            frame_bounding_box_count.append(bounding_box_count)

            # 이전에 그려진 모든 트랙 그리기
            for track in all_tracks:
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(frame, [points], isClosed=False, color=(0, 0, 255), thickness=1)

            # 현재 프레임의 트랙을 저장
            all_tracks.extend(track_history.values())

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_pil = Image.fromarray(frame_rgb)
            frame_photo = ImageTk.PhotoImage(image=frame_pil)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=frame_photo)
            self.canvas.photo = frame_photo

            self.out.write(frame)

            if ret:
                self.root.after(30, self.update_canvas)
            else:
                self.cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # 영상을 다시 처음으로 돌아가게 함
                self.update_canvas()  # 다시 재생

    def update_right_canvas(self):
        if self.right_playing:
            ret, frame = self.right_video.read()
            if ret:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_pil = Image.fromarray(frame_rgb)
                frame_photo = ImageTk.PhotoImage(image=frame_pil)
                self.canvas_right.create_image(0, 0, anchor=tk.NW, image=frame_photo)
                self.canvas_right.photo = frame_photo
            else:
                # 영상이 끝까지 갔을 때 다시 처음으로 돌아감
                self.right_video.set(cv2.CAP_PROP_POS_FRAMES, 0)
            if self.right_playing:
                self.root.after(30, self.update_right_canvas)  # 오른쪽 영상 업데이트

    def toggle_left_play(self):
        self.left_playing = not self.left_playing
        if self.left_playing:
            self.update_canvas()

    def toggle_right_play(self):
        self.right_playing = not self.right_playing
        if self.right_playing:
            self.update_right_canvas()

if __name__ == "__main__":
    root = tk.Tk()
    app = VideoPlayerApp(root)
    root.mainloop()

WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 103 Spermss, 18.9ms
Speed: 2.0ms preprocess, 18.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 99 Spermss, 18.9ms
Speed: 1.0ms preprocess, 18.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 101 Spermss, 17.9ms
Speed: 2.0ms preprocess, 17.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 104 Spermss, 18.9ms
Speed: 1.0


0: 480x640 104 Spermss, 27.9ms
Speed: 1.9ms preprocess, 27.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 107 Spermss, 28.9ms
Speed: 1.9ms preprocess, 28.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 108 Spermss, 28.8ms
Speed: 1.1ms preprocess, 28.8ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 107 Spermss, 28.9ms
Speed: 2.0ms preprocess, 28.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be remo

WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 106 Spermss, 38.8ms
Speed: 2.0ms preprocess, 38.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 110 Spermss, 38.9ms
Speed: 2.0ms preprocess, 38.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 109 Spermss, 38.9ms
Speed: 2.0ms preprocess, 38.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 109 Spermss, 39.9ms
Speed: 0.


0: 480x640 112 Spermss, 17.9ms
Speed: 1.0ms preprocess, 17.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 112 Spermss, 18.9ms
Speed: 1.0ms preprocess, 18.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 110 Spermss, 17.9ms
Speed: 1.0ms preprocess, 17.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 112 Spermss, 17.9ms
Speed: 2.0ms preprocess, 17.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be remo

WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 113 Spermss, 24.9ms
Speed: 2.0ms preprocess, 24.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 111 Spermss, 26.9ms
Speed: 1.0ms preprocess, 26.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 111 Spermss, 25.9ms
Speed: 2.0ms preprocess, 25.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 73 Spermss, 26.9ms
Speed: 1.0

...

WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 112 Spermss, 39.9ms
Speed: 1.0ms preprocess, 39.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 110 Spermss, 39.9ms
Speed: 2.0ms preprocess, 39.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 114 Spermss, 39.9ms
Speed: 1.0ms preprocess, 39.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 113 Spermss, 39.8ms
Speed: 1.


0: 480x640 111 Spermss, 38.9ms
Speed: 2.0ms preprocess, 38.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 73 Spermss, 38.9ms
Speed: 1.0ms preprocess, 38.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 99 Spermss, 38.9ms
Speed: 0.9ms preprocess, 38.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 98 Spermss, 38.9ms
Speed: 1.0ms preprocess, 38.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed

WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 105 Spermss, 38.9ms
Speed: 1.9ms preprocess, 38.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 107 Spermss, 52.8ms
Speed: 2.0ms preprocess, 52.8ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 108 Spermss, 51.8ms
Speed: 1.9ms preprocess, 51.8ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 108 Spermss, 49.8ms
Speed: 1.


0: 480x640 111 Spermss, 39.8ms
Speed: 1.0ms preprocess, 39.8ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 106 Spermss, 38.9ms
Speed: 2.0ms preprocess, 38.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 110 Spermss, 39.9ms
Speed: 1.0ms preprocess, 39.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be removed in 'ultralytics 8.299999999999999' in the future. Please use 'show_labels' instead.

0: 480x640 109 Spermss, 47.8ms
Speed: 1.0ms preprocess, 47.8ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
WARNING ⚠️ 'hide_labels' is deprecated and will be remo

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\USER\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\USER\AppData\Local\Temp\ipykernel_4656\1873462806.py", line 187, in toggle_right_play
    self.update_right_canvas()
  File "C:\Users\USER\AppData\Local\Temp\ipykernel_4656\1873462806.py", line 166, in update_right_canvas
    ret, frame = self.right_video.read()
AttributeError: 'NoneType' object has no attribute 'read'
